In [ ]:
import zipfile

zip_ref = zipfile.ZipFile("all.zip", 'r')
zip_ref.extractall(".")
zip_ref.close()

zip_ref = zipfile.ZipFile("train.zip", 'r')
zip_ref.extractall(".")
zip_ref = zipfile.ZipFile("test.zip", 'r')
zip_ref.extractall(".")
zip_ref.close()

In [1]:
from keras.models import Model
from keras.layers import Dense, Input, GlobalAveragePooling2D, Dropout, Lambda
from keras.applications import inception_resnet_v2
# from keras.preprocessing.image import *

import numpy as np
from tqdm import tqdm
import cv2

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
n = 25000
img_height, img_width = 299, 299
X = np.zeros((n, img_height, img_width, 3), dtype=np.uint8)
y = np.zeros((n, 1), dtype=np.uint8)

for i in tqdm(range(12500)): # n/2
    X[i] = cv2.resize(cv2.imread('train/cat.%d.jpg' % i), (img_height, img_width))
    X[i+12500] = cv2.resize(cv2.imread('train/dog.%d.jpg' % i), (img_height, img_width)) # n/2

# 打上标签
y[12500:] = 1

100%|██████████| 12500/12500 [03:56<00:00, 52.81it/s]


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

In [7]:
i_input_tensor = Input((img_height, img_width, 3))
# x_input_tensor = inception_resnet_v2.preprocess_input(i_input_tensor)	# 错误的处理
x_input_tensor = Lambda(inception_resnet_v2.preprocess_input)(i_input_tensor) # 建立模型预处理

# 搭建基础模型
base_model = inception_resnet_v2.InceptionResNetV2(input_tensor=x_input_tensor, weights='imagenet', include_top=False)

In [ ]:
范例：不运行
from keras.callbacks import ModelCheckpoint

'''
如果验证损失下降， 那么在每个训练轮之后保存模型。
'''
checkpointer = ModelCheckpoint(filepath='weights.hdf5', verbose=1, save_best_only=True)
model.fit(x_train, y_train, batch_size=128, epochs=20, verbose=0, validation_data=(X_test, Y_test), callbacks=[checkpointer])

In [9]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='weights_checkpint.hdf5', verbose=1, save_best_only=True)

In [20]:
# 锁定所有神经网络层
for layers in base_model.layers:
    layers.trainable = False

i_output = GlobalAveragePooling2D()(base_model.output)
i_output = Dropout(0.5)(i_output) # 0.25 比 0.5 好
i_predictions = Dense(1, activation='sigmoid')(i_output)
model = Model(base_model.input, i_predictions)

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [21]:
model.fit(X_train, y_train, batch_size=20,     # defalt 16
	epochs=8, 	# defalt 5
	validation_data=(X_valid, y_valid),
         verbose=1, callbacks=[checkpointer])


Train on 20000 samples, validate on 5000 samples
Epoch 1/8
20000/20000 [==============================] - 156s 8ms/step - loss: 0.1433 - acc: 0.9508 - val_loss: 0.0386 - val_acc: 0.9904

Epoch 00001: val_loss did not improve from 0.03454
Epoch 2/8
20000/20000 [==============================] - 146s 7ms/step - loss: 0.0923 - acc: 0.9651 - val_loss: 0.0425 - val_acc: 0.9888

Epoch 00002: val_loss did not improve from 0.03454
Epoch 3/8
20000/20000 [==============================] - 147s 7ms/step - loss: 0.0846 - acc: 0.9676 - val_loss: 0.0325 - val_acc: 0.9912

Epoch 00003: val_loss improved from 0.03454 to 0.03248, saving model to weights_checkpint.hdf5
Epoch 4/8
20000/20000 [==============================] - 147s 7ms/step - loss: 0.0766 - acc: 0.9707 - val_loss: 0.0396 - val_acc: 0.9896

Epoch 00004: val_loss did not improve from 0.03248
Epoch 5/8
20000/20000 [==============================] - 147s 7ms/step - loss: 0.0804 - acc: 0.9680 - val_loss: 0.0218 - val_acc: 0.9948

Epoch 00005: 

In [22]:
for i in range(len(model.layers)):
    print(model.layers[i].name, i)

input_2 0
lambda_2 1
conv2d_204 2
batch_normalization_204 3
activation_204 4
conv2d_205 5
batch_normalization_205 6
activation_205 7
conv2d_206 8
batch_normalization_206 9
activation_206 10
max_pooling2d_5 11
conv2d_207 12
batch_normalization_207 13
activation_207 14
conv2d_208 15
batch_normalization_208 16
activation_208 17
max_pooling2d_6 18
conv2d_212 19
batch_normalization_212 20
activation_212 21
conv2d_210 22
conv2d_213 23
batch_normalization_210 24
batch_normalization_213 25
activation_210 26
activation_213 27
average_pooling2d_2 28
conv2d_209 29
conv2d_211 30
conv2d_214 31
conv2d_215 32
batch_normalization_209 33
batch_normalization_211 34
batch_normalization_214 35
batch_normalization_215 36
activation_209 37
activation_211 38
activation_214 39
activation_215 40
mixed_5b 41
conv2d_219 42
batch_normalization_219 43
activation_219 44
conv2d_217 45
conv2d_220 46
batch_normalization_217 47
batch_normalization_220 48
activation_217 49
activation_220 50
conv2d_216 51
conv2d_218 52
c

In [23]:
# 放开靠后部分网络层进行 fine tune 微调模型
for layer in model.layers[770:]:	# total 783 layers
    layer.trainable = True


from keras import optimizers
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])


In [24]:
model.fit(X_train, y_train, batch_size=20, 	# first: 16
	epochs=8, 	# first: 5 
	validation_data=(X_valid, y_valid),
         verbose=1, callbacks=[checkpointer])

Train on 20000 samples, validate on 5000 samples
Epoch 1/8
20000/20000 [==============================] - 160s 8ms/step - loss: 0.0760 - acc: 0.9726 - val_loss: 0.0369 - val_acc: 0.9894

Epoch 00001: val_loss did not improve from 0.02176
Epoch 2/8
20000/20000 [==============================] - 150s 8ms/step - loss: 0.0742 - acc: 0.9722 - val_loss: 0.0372 - val_acc: 0.9894

Epoch 00002: val_loss did not improve from 0.02176
Epoch 3/8
20000/20000 [==============================] - 150s 8ms/step - loss: 0.0731 - acc: 0.9720 - val_loss: 0.0375 - val_acc: 0.9890

Epoch 00003: val_loss did not improve from 0.02176
Epoch 4/8
20000/20000 [==============================] - 150s 8ms/step - loss: 0.0723 - acc: 0.9732 - val_loss: 0.0368 - val_acc: 0.9892

Epoch 00004: val_loss did not improve from 0.02176
Epoch 5/8
20000/20000 [==============================] - 150s 8ms/step - loss: 0.0737 - acc: 0.9725 - val_loss: 0.0368 - val_acc: 0.9894

Epoch 00005: val_loss did not improve from 0.02176
Epoch 

In [25]:
model.save('modelInceptionResNet_transfer_Doup050_770layer.h5')

读取测试图片数据

In [26]:
X_test = np.zeros((n, img_height, img_width, 3), dtype=np.uint8)
for i in tqdm(range(12500)):
    j = i+1
    X_test[i] = cv2.resize(cv2.imread('test/%d.jpg' % j), (img_height, img_width))

100%|██████████| 12500/12500 [00:28<00:00, 443.37it/s]


In [27]:
y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

25000/25000 [==============================] - 151s 6ms/step


In [28]:
import pandas as pd
df = pd.read_csv("sample_submission.csv")

for i in range(12500):
	df.set_value(i, 'label', y_pred[i])

df.to_csv('pred_changePredictCsvMethond_Drop050_770layer.csv', index=None)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
